In [1]:
# tampering with the kwd file on passaro
# the /experiment folder is on the ssd of passaro (/usr/local/experiment/raw_data)
%matplotlib inline
from __future__ import division
import pdb
import sys
import pandas as pd
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import matplotlib
import scipy.signal as sg
import math
import scipy as sp
import socket
import os
import wave
import struct
import h5py
from scipy.signal import hilbert
from basic_viewing import h5_functions as h5
import glob
import errno    
import os
import shutil as sh
import yaml

matplotlib.style.use('ggplot')

# Check wihic computer to decide where the things are mounted
comp_name=socket.gethostname()
print 'Computer: ' + comp_name
if  comp_name == 'chim':
    #sys.path.append('/Users/zeke/experiment/ephysDataManagement/ephysScripts')
    experiment_folder = os.path.join(os.path.abspath('W:') , "earneodo", "bci_zf")
elif  'lookfar' in comp_name:
    repos_folder = os.path.abspath('/Users/zeke/reposs')
    experiment_folder = os.path.join(os.path.abspath('/Volumes'), 
                                     'gentner', 
                                     'earneodo', 
                                     'bci_zf')

elif 'lintu' in comp_name:
    repos_folder = os.path.abspath('/mnt/cube/earneodo/repos')
    experiment_folder = os.path.join('/mnt/cube/earneodo/bci_zf/')

elif 'passaro' in comp_name:
    repos_folder = os.path.abspath('/mnt/cube/earneodo/repos')
    cube_experiment_folder = os.path.join('/mnt/cube/earneodo/bci_zf/')
    store_experiment_folder = os.path.join('/Data/bci_zf/')
    experiment_folder = os.path.join('/usr/local/experiment')

sys.path.append(os.path.join(repos_folder, 'soundflow', 'sound_tools'))
sys.path.append(os.path.join(repos_folder, 'ephysflow'))
sys.path.append(os.path.join(repos_folder, 'analysis-tools'))

import soundtools as st
import Kwik as oe
from scipy.io import wavfile
from file_tools import experiment as et

def create_neural_data_set(data_set, parent_group, channel_list, 
                           frame_size=None, 
                           processing=None,
                           *args, **kwargs):
    # make the new dataset
    d_type = data_set.dtype
    nd_cols = np.array(channel_list).size
    nd_rows = data_set.shape[0]
    d_chunks = np.array(data_set.chunks)
    
    if d_chunks[0] > nd_rows:
        d_chunks[0] = nd_rows
    
    if frame_size is None:
        frame_size = d_chunks[0] if d_chunks[0]<nd_rows else nd_rows
    
    neural_dset = parent_group.create_dataset("data", (nd_rows, nd_cols), 
                                              chunks=(d_chunks[0], nd_cols), dtype=d_type)
    
    copy_frame = np.zeros([frame_size, nd_cols], dtype=d_type)
    n_full_frames = int(data_set.shape[0]/frame_size)
    #print neural_dset.shape
    # Fill the dataset:
    for i in range(n_full_frames):
        copy_frame = h5.load_table_slice(data_set, 
                                         np.arange(i*frame_size, (i+1)*frame_size), 
                                         channel_list)
        neural_dset[i*frame_size: (i+1)*frame_size, 0:nd_cols] = copy_frame
    
    if frame_size*n_full_frames < nd_rows:
        last_frame_size = nd_rows - frame_size*n_full_frames
        copy_frame = np.zeros([last_frame_size, nd_cols], dtype=d_type)
        copy_frame = h5.load_table_slice(data_set, 
                                         np.arange(n_full_frames*frame_size, 
                                                   n_full_frames*frame_size + last_frame_size),
                                         channel_list)
    # copy the attributes:
    neural_dset.attrs.create('valid_samples', 
                             np.array(data_set.attrs['valid_samples'][channel_list]))
    
def copy_attribs(source, dest):
    for key, attrib in source.attrs.iteritems():
        dest.attrs.create(key, attrib, dtype=attrib.dtype)
        
def copy_application_data(source_rec, dest_rec, chan_list,
                          resize_keys = ['channel_bit_volts', 'channel_sample_rates']):
    
    dest_rec.create_group('application_data')
    for key, attrib in source_rec['application_data'].attrs.iteritems():
        if not key in resize_keys:
            dest_rec['application_data'].attrs.create(key, attrib, dtype=attrib.dtype)
        else:
            #print key
            #print attrib[chan_list]
            dest_rec['application_data'].attrs.create(key, attrib[chan_list], dtype=attrib.dtype)

def check_rec_data(source_rec):
    #check the data table of a rec group
    data_set = source_rec['data']
    valid_samples = None
    try:
        valid_samples = data_set.attrs.get('valid_samples')
    except:
        pass
    
    has_data = True if valid_samples is not None and valid_samples[0]>0 else False
    return has_data

def get_valid_samples(source_rec):
    #get the valid samples declared by the data table attributes
    data_set = source_rec['data']
    valid_samples = None
    try:
        valid_samples = data_set.attrs.get('valid_samples')[0]
    except:
        pass
    return valid_samples

def create_data_groups(raw_file, new_file, chan_list):
    for rec_name, rec_group in raw_file['/recordings'].iteritems():
        new_file['/recordings'].create_group(rec_name)
        copy_attribs(rec_group, new_file['/recordings'][rec_name])
        copy_application_data(rec_group, new_file['/recordings'][rec_name], chan_list)
        create_neural_data_set(rec_group['data'], new_file['/recordings'][rec_name], chan_list)
        
def make_shank_kwd(raw_file, out_file_path, chan_list):
    ss_file = h5py.File(out_file_path, 'w')
    copy_attribs(raw_file, ss_file)
    ss_file.create_group('/recordings')
    create_data_groups(raw_file, ss_file, chan_list) 
    ss_file.close()
    
def list_flatten(x):
    return [val for sublist in x for val in sublist]

def mkdir_p(path):
    try:
        os.makedirs(path)
    except OSError as exc:  # Python >2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise

def make_super_file(path):
    new_file = h5py.File(path, 'w')
    new_file.create_group('/recordings')
    new_file.close()

def create_data_group(raw_file, new_file, chan_list, new_rec_name):
    for rec_name, rec_group in raw_file['/recordings'].iteritems():
        new_file['/recordings'].create_group(rec_name)
        copy_attribs(rec_group, new_file['/recordings'][rec_name])
        copy_application_data(rec_group, new_file['/recordings'][rec_name], chan_list)
        create_neural_data_set(rec_group['data'], new_file['/recordings'][rec_name], chan_list)

def insert_neural_rec_group(dest_file, raw_rec_group, chan_list, new_group_name=None):
    rec_name = raw_rec_group.name if new_group_name is None else new_group_name
    
    dest_file['/recordings'].create_group(rec_name)
    new_rec = dest_file['/recordings'][rec_name]
    copy_application_data(raw_rec_group, new_rec, chan_list)
    copy_attribs(raw_rec_group, new_rec)
    create_neural_data_set(raw_rec_group['data'], new_rec, chan_list)
    
def modify_rec_group_attribs(kwd_file, rec_name, attr_dict, new_attr_dict = None):
    for key, value in attr_dict.iteritems():
        kwd_file['/recordings'][rec_name].attrs.modify(key, value)
    
    if new_attr_dict is not None:
        for key, value in new_attr_dict.iteritems():
            kwd_file['/recordings'][rec_name].attrs.create(key, value)

def get_recs_list(experiment_file):
    if experiment_file['/recordings'].keys() == []:
        rec_list = None
    else:
        rec_list = np.array([int(i) for i in experiment_file['/recordings'].keys()])
    return rec_list

def get_experiment_endpoints(experiment_file):
    recs_list = get_recs_list(experiment_file)
    if recs_list is not None:
        last_rec_num = np.max(recs_list)
        last_rec_name = str(last_rec_num)
        last_rec = experiment_file['/recordings'][last_rec_name]
        last_rec_start = last_rec.attrs['start_sample']
        last_rec_nsamples = last_rec['data'].shape[0]
        next_rec_num = last_rec_num + 1
        next_sample = last_rec_start + last_rec_nsamples
    else:
        next_rec_num = 0
        next_sample = 0
        
    return [next_rec_num, next_sample]
            

def insert_experiment_groups(dest_file, raw_file, chan_list):
    # all the recs in an experiment file if they pass check
    for raw_rec_name, raw_rec_group in raw_file['/recordings'].iteritems():
        #print 'rec {0}'.format(raw_rec_name)
        s_f = raw_rec_group.attrs['sample_rate']
        if check_rec_data(raw_rec_group):
            target_rec_num, target_start_sample = get_experiment_endpoints(dest_file)
            target_rec_name = str(target_rec_num)
            target_start_time = int(target_start_sample/(0.001*s_f))
            target_source = '{0}:''/''recordings/{1}'.format(raw_file.filename, raw_rec_name)
            insert_neural_rec_group(dest_file, raw_rec_group, chan_list, new_group_name=target_rec_name)
            modify_rec_group_attribs(dest_file, target_rec_name, 
                             {'start_sample' : target_start_sample},
                              new_attr_dict = {'name' : target_source,
                                               'start_time' : target_start_time})
        else:
            #print "Skipping rec {0} with no data".format(raw_rec_name)
            pass

def band_pass_filter_pars(s_f, exp_par):
    filt_hi = exp_par['search_motiff']['filt_hi']
    filt_lo = exp_par['search_motiff']['filt_lo']
    hp_b, hp_a = sg.butter(4, filt_hi/(s_f/2.), btype='high')
    lp_b, lp_a = sg.butter(4, filt_lo/(s_f/2.), btype='low')
    return hp_b, hp_a, lp_b, lp_a

def filter_bp(data, hp_b, hp_a, lp_b, lp_a):
    x_hi = sg.filtfilt(hp_b, hp_a, data, axis=0)
    x_lo = sg.filtfilt(lp_b, lp_a, x_hi, axis=0)
    return x_lo

def get_dset_group_attr(data_set, attr_name):
    return data_set.parent.attrs[attr_name]

def export_audio(data_set, chan_number, out_file_path, filter_func=None, args=(), **kwargs):
    frame_size = 60 #seconds
    s_f = get_dset_group_attr(data_set, 'sample_rate')
    samples_frame = int(frame_size * s_f)
    samples_data = data_set.shape[0]
    
    frame_buffer = np.zeros((samples_frame, 1), dtype = np.float)
    frame_starts = np.arange(0, samples_data, samples_frame)
    
    wavefile = wave.open(out_file_path,'wb')
    wavefile.setparams((1, 2, s_f, 0, 'NONE', 'not compressed'))
    
    for start in frame_starts:
        end = min(start + samples_frame, samples_data)
        frame_buffer[0: end-start, :] = h5.load_table_slice(data_set, 
                                                         np.arange(start, end),
                                                         [chan_number])
        frame_buffer[0:end-start, :] = filter_func(frame_buffer[0:end-start, :], 
                                                  *args, **kwargs)
        wavefile.writeframes(frame_buffer[0: end-start].astype('h').tostring())
    
    wavefile.close()

Computer: passaro


In [3]:
bird_id = 'z017'
sess_day = '2016-07-13'
depth = 300

channel_config = {'neural': [0, 11, 9, 16, 14, 5, 8, 10, 15, 2, 7, 4, 3, 5, 1, 16, 15],
                  'super' : range(17),
                  '0' : [11, 15, 8, 10],
                  '1' : [14, 9, 13, 12],
                  '2' : [2, 3, 1, 6],
                  '3' : [5, 7, 0, 4], 
                  'test' : [11],
                  'mic': [16]}

chan_list= channel_config['super']
# Make the super session and gather the list of sessions
# get the sessions list of files
fn = et.file_names(bird_id, sess_day)

raw_data_folder = os.path.join(experiment_folder, 'raw_data')
ss_data_folder = os.path.join(cube_experiment_folder, 'ss_data')

sessions = glob.glob(os.path.join(raw_data_folder, bird_id, sess_day + '*' + str(depth)))
experiments = list_flatten([ glob.glob(os.path.join(s, '*.raw.kwd'))[:] for s in sessions ])
experiments.sort()
sessions = glob.glob(os.path.join(raw_data_folder, bird_id, sess_day + '*' + str(depth)))
print sessions

[]


In [4]:
# get the parameter file of the first rec in the day
super_sess_name = 'day-' + sess_day
fn = et.file_names(bird_id, super_sess_name, 0)
super_sess_path = fn['folders']['ss']
super_file_path = os.path.join(super_sess_path, fn['files']['ss_raw'])

sess_par = et.get_parameters(bird_id, os.path.split(os.path.split(experiment_path)[0])[1], 
                            location='raw')

NameError: name 'experiment_path' is not defined

In [23]:
bird_id = 'z017'
sess_day = '2016-07-12'
depth = 225

channel_config = {'neural': [0, 11, 9, 16, 14, 5, 8, 10, 15, 2, 7, 4, 3, 5, 1, 16, 15],
                  'super' : range(17),
                  '0' : [11, 15, 8, 10],
                  '1' : [14, 9, 13, 12],
                  '2' : [2, 3, 1, 6],
                  '3' : [5, 7, 0, 4], 
                  'test' : [11],
                 'sound': [17]}

chan_list= channel_config['super']
# Make the super session and gather the list of sessions
# get the sessions list of files
sessions = glob.glob(os.path.join(raw_data_folder, bird_id, sess_day + '*' + str(depth)))
experiments = list_flatten([ glob.glob(os.path.join(s, '*.raw.kwd'))[:] for s in sessions ])
experiments.sort()

# make the supersession file
super_sess_name = 'day-' + sess_day
fn = et.file_names(bird_id, super_sess_name, 0)
super_sess_path = fn['folders']['ss']
super_file_path = os.path.join(super_sess_path, fn['files']['ss_raw'])

print "Making supersession {}".format(super_sess_name)
print super_file_path
mkdir_p(super_sess_path)
super_file = h5py.File(super_file_path, 'a')
exp_par_file = 
for experiment_path in experiments:
    print 'File {0}'.format(experiment_path)
    sess_fold = os.path.split(os.path.split(experiment_path)[0])[1]
    print sess_fold
    sess_par = et.get_parameters(bird_id, sess_fold, location='raw')


Making supersession day-2016-07-13
/mnt/cube/earneodo/bci_zf/ss_data/z017/day-2016-07-13/experiment.raw.kwd


In [27]:
day_par_file_path = os.path.join(sessions[0], fn['files']['par'])
sh.copy2(day_par_file_path, super_sess_path)

'/usr/local/experiment/raw_data/z017/2016-07-13_17-23-17_300/experiment.par.yml'

In [33]:
sess_par['channel_config']['neural'] + [sess_par['channel_config']['mic'], ]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

In [34]:
sess_par['channel_config']

{'mic': 16, 'neural': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]}

In [9]:
bird_id = 'z017'
sess_day = '2016-07-13'
depth = 300

# Make the super session and gather the list of sessions
# get the sessions list of files
sessions = glob.glob(os.path.join(raw_data_folder, bird_id, sess_day + '*' + str(depth)))
experiments = list_flatten([ glob.glob(os.path.join(s, '*.raw.kwd'))[:] for s in sessions ])
experiments.sort()

# make the supersession file
super_sess_name = 'day-' + sess_day
fn = et.file_names(bird_id, super_sess_name, 0)
super_sess_path = fn['folders']['ss']
super_file_path = os.path.join(super_sess_path, fn['files']['ss_raw'])

print "Making supersession {}".format(super_sess_name)
print super_file_path
mkdir_p(super_sess_path)
print "Copying parameter file: "
day_par_file_path = os.path.join(sessions[0], fn['files']['par'])
sh.copy2(day_par_file_path, super_sess_path)
print "making the superfile"
make_super_file
super_file = h5py.File(super_file_path, 'a')
for experiment_path in experiments:
    print 'File {0}'.format(experiment_path)
    sess_fold = os.path.split(os.path.split(experiment_path)[0])[1]
    print sess_fold
    sess_par = et.get_parameters(bird_id, sess_fold, location='raw')
    chan_list = sess_par['channel_config']['neural'] + [sess_par['channel_config']['mic'], ]
    raw_file = h5py.File(experiment_path, 'r')
    insert_experiment_groups(super_file, raw_file, chan_list)
    raw_file.close()
    super_file.flush()

super_file.close()

#make the backups
bkp_dest_path = os.path.join(cube_experiment_folder, 'raw_data', bird_id)
store_dest_path = os.path.join(store_experiment_folder, 'raw_data', bird_id)

mkdir_p(bkp_dest_path)
mkdir_p(store_dest_path)
for session_path in sessions:
    session_name = os.path.split(session_path)[-1]
    print "cube backup: " + session_name
    dest_bkp = os.path.join(bkp_dest_path, session_name)
    try:
        sh.copytree(session_path, dest_bkp)
    except OSError as exc:  # Python >2.5
        if exc.errno == 17:
            pass
        else:
            raise
            
    print "passaro Data backup: " + session_name
    sh.move(session_path, store_dest_path)
    
extract_wav_chans(bird_id, sess_day)
print "done"

Making supersession day-2016-07-12
/mnt/cube/earneodo/bci_zf/ss_data/z017/day-2016-07-12/experiment.raw.kwd
File /usr/local/experiment/raw_data/z017/2016-07-12_09-45-21_225/experiment1_100.raw.kwd
2016-07-12_09-45-21_225
File /usr/local/experiment/raw_data/z017/2016-07-12_11-01-09_225/experiment1_100.raw.kwd
2016-07-12_11-01-09_225
File /usr/local/experiment/raw_data/z017/2016-07-12_11-01-09_225/experiment2_100.raw.kwd
2016-07-12_11-01-09_225
File /usr/local/experiment/raw_data/z017/2016-07-12_13-12-09_225/experiment1_100.raw.kwd
2016-07-12_13-12-09_225
File /usr/local/experiment/raw_data/z017/2016-07-12_13-44-59_225/experiment1_100.raw.kwd
2016-07-12_13-44-59_225
cube backup: 2016-07-12_13-44-59_225
passaro Data backup: 2016-07-12_13-44-59_225
cube backup: 2016-07-12_11-01-09_225
passaro Data backup: 2016-07-12_11-01-09_225
cube backup: 2016-07-12_13-12-09_225
passaro Data backup: 2016-07-12_13-12-09_225
cube backup: 2016-07-12_09-45-21_225
passaro Data backup: 2016-07-12_09-45-21_225

In [11]:
def extract_wav_chans(bird, sess, ch_name='mic'):
    fn = et.file_names(bird, sess, 0)
    super_file_path = os.path.join(fn['folders']['ss'], fn['files']['ss_raw'])
    exp_par = et.get_parameters(bird_id, super_sess_name)
    super_file = h5py.File(super_file_path, 'r')
    rec_list = super_file['/recordings'].keys()
    print rec_list

    for rec, rec_grp in super_file['/recordings'].iteritems():
        print rec
        fn = et.file_names(bird, sess, int(rec))
        data_set = rec_grp['data']
        s_f = rec_grp.attrs['sample_rate']
        wav_file_path = os.path.join(fn['folders']['ss'], fn['files'][ch_name])
        print wav_file_path
        export_audio(data_set, exp_par['channel_config'][ch_name], wav_file_path,
                    filter_func=filter_bp,
                    args=band_pass_filter_pars(s_f, exp_par))


extract_wav_chans('z020', 'day-sleep_remote-2016-06-05')

[u'0', u'1', u'10', u'100', u'101', u'102', u'103', u'104', u'105', u'106', u'107', u'108', u'109', u'11', u'110', u'111', u'112', u'113', u'114', u'115', u'116', u'117', u'118', u'119', u'12', u'120', u'121', u'122', u'123', u'124', u'125', u'126', u'127', u'128', u'129', u'13', u'130', u'131', u'132', u'133', u'134', u'135', u'136', u'137', u'138', u'139', u'14', u'140', u'141', u'142', u'143', u'144', u'145', u'146', u'147', u'148', u'149', u'15', u'150', u'151', u'152', u'153', u'154', u'155', u'156', u'157', u'158', u'159', u'16', u'160', u'161', u'162', u'163', u'164', u'165', u'166', u'167', u'168', u'169', u'17', u'170', u'171', u'172', u'173', u'174', u'18', u'19', u'2', u'20', u'21', u'22', u'23', u'24', u'25', u'26', u'27', u'28', u'29', u'3', u'30', u'31', u'32', u'33', u'34', u'35', u'36', u'37', u'38', u'39', u'4', u'40', u'41', u'42', u'43', u'44', u'45', u'46', u'47', u'48', u'49', u'5', u'50', u'51', u'52', u'53', u'54', u'55', u'56', u'57', u'58', u'59', u'6', u'60', 